In [1]:
import nidaqmx
from nidaqmx.constants import AcquisitionType

### Simple CI (no time-bi saving, just calculating total number of edges)

In [603]:
task = nidaqmx.Task()
task.ci_channels.add_ci_count_edges_chan("/PXI-6363/ctr0")
task.start()

In [623]:
task.read()

21

In [624]:
task.stop()

### CI: counte edges, continuous, buffered

In [394]:
task = nidaqmx.Task()
task.ci_channels.add_ci_count_edges_chan("/PXI-6363/ctr0")
task.timing.cfg_samp_clk_timing(
    rate=int(10),
    source='/PXI-6363/PFI12',
    sample_mode=AcquisitionType.CONTINUOUS,
    samps_per_chan=1000
)
task.start()

External SampleClock signal is required

In [400]:
task.read(nidaqmx.constants.READ_ALL_AVAILABLE)

[16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18]

In [393]:
task.stop()
task.close()

In [401]:
task.read(nidaqmx.constants.READ_ALL_AVAILABLE)

DaqError: The application is not able to keep up with the hardware acquisition.

Increasing the buffer size, reading the data more frequently, or specifying a fixed number of samples to read instead of reading all available samples might correct the problem.
Property: DAQmx_Read_RelativeTo
Requested Value: DAQmx_Val_CurrReadPos

Property: DAQmx_Read_Offset
Requested Value: 0

Task Name: _unnamedTask<13>

Status Code: -200279

In [402]:
task.stop()
task.close()

### CO: generate waveform

In [404]:
help(nidaqmx.Task().co_channels.add_)

Help on method add_co_pulse_chan_freq in module nidaqmx._task_modules.co_channel_collection:

add_co_pulse_chan_freq(counter, name_to_assign_to_channel='', units=<FrequencyUnits.HZ: 10373>, idle_state=<Level.LOW: 10214>, initial_delay=0.0, freq=1.0, duty_cycle=0.5) method of nidaqmx._task_modules.co_channel_collection.COChannelCollection instance
    Creates channel(s) to generate digital pulses that **freq** and
    **duty_cycle** define. The pulses appear on the default output
    terminal of the counter unless you select a different output
    terminal.
    
    Args:
        counter (str): Specifies the names of the counters to use to
            create the virtual channels. The DAQmx physical channel
            constant lists all physical channels, including
            counters, for devices installed in the system.
        name_to_assign_to_channel (Optional[str]): Specifies a name
            to assign to the virtual channel this function creates.
            If you do not spec

In [3]:
task = nidaqmx.Task()
task.co_channels.add_co_pulse_chan_freq(counter='/PXI-6363/ctr1', freq=1000, duty_cycle=0.5)
task.timing.cfg_implicit_timing(sample_mode=AcquisitionType.CONTINUOUS)

In [4]:
task.start()

In [5]:
task.stop()
task.close()

### CI: using 2nd counter as the SampleClock signal generator

In [592]:
clock_task = nidaqmx.Task()
clock_task.co_channels.add_co_pulse_chan_freq(counter='/PXI-6363/ctr1', freq=10, duty_cycle=0.5)
clock_task.timing.cfg_implicit_timing(sample_mode=AcquisitionType.CONTINUOUS)

In [593]:
clock_task.start()

In [600]:
clock_task.stop()
clock_task.close()

In [596]:
ci_task = nidaqmx.Task()
ci_task.ci_channels.add_ci_count_edges_chan("/PXI-6363/ctr0")
ci_task.timing.cfg_samp_clk_timing(
    rate=int(10),
    source='/PXI-6363/Ctr1InternalOutput',
    sample_mode=AcquisitionType.CONTINUOUS,
    samps_per_chan=1000
)

In [597]:
ci_task.start()

In [598]:
ci_task.read(nidaqmx.constants.READ_ALL_AVAILABLE)

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5]

In [602]:
ci_task.stop()
ci_task.close()